There is the simplest models always predicting a cosnstant for `updrs_1, updrs_2, updrs_3, updrs_4` [Only Constants](https://www.kaggle.com/code/vitalykudelya/only-constants).

Let's complicate the previous model and add trend for the `pred_month` feature (from a model with one paramter we moved to model with two parameters).

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px

import amp_pd_peptide

from scipy.optimize import minimize

In [2]:
def smape_plus_1(y_true, y_pred):
    y_true_plus_1 = y_true + 1
    y_pred_plus_1 = y_pred + 1
    metric = np.zeros(len(y_true_plus_1))
    
    numerator = np.abs(y_true_plus_1 - y_pred_plus_1)
    denominator = ((np.abs(y_true_plus_1) + np.abs(y_pred_plus_1)) / 2)
    
    mask_not_zeros = (y_true_plus_1 != 0) | (y_pred_plus_1 != 0)
    metric[mask_not_zeros] = numerator[mask_not_zeros] / denominator[mask_not_zeros]
    
    return 100 * np.nanmean(metric)

In [3]:
train_clinical_data = pd.read_csv('/kaggle/input/amp-parkinsons-disease-progression-prediction/train_clinical_data.csv')
train_clinical_data['source'] = 'standard'

supplemental_clinical_data = pd.read_csv('/kaggle/input/amp-parkinsons-disease-progression-prediction/supplemental_clinical_data.csv')
supplemental_clinical_data['source'] = 'supplemental'

train_clinical_all = pd.concat([train_clinical_data, supplemental_clinical_data])

## Plots of y_true

In [4]:
for target in ['updrs_1', 'updrs_2', 'updrs_3', 'updrs_4']:
    fig = px.box(
        train_clinical_all, 
        y=target, 
        x='visit_month', 
        color='source', 
        title=f'{target} per visit_month'
    )
    fig.show()
    
    fig = px.histogram(
        train_clinical_all, 
        x=target, 
        color='source', 
        title=f'{target} Distribution', 
        histnorm='probability', 
        barmode='overlay'
    )
    fig.show()

It's interesting that for supplemental_data `max(visit_month) = 36`, for standard data we have much more visit_months. Distributions of **y_true** are also different. <br>
Additionally we cand find a little trend. <br>

In [5]:
# delete visit_month 3, 5, 9 (there are no such visit_months in the Test API)
train_clinical_all = train_clinical_all[~train_clinical_all.visit_month.isin([3, 5, 9])]

## Create Train Dataset

More detailed information why we use only plus 6, 12, 24 months of **y_true** and other mechanics of this competition could be found in the notebook [Explain Dataset, Test API, Cross-Validation Tips](https://www.kaggle.com/code/vitalykudelya/explain-dataset-test-api-cross-validation-tips).

In [6]:
train_clinical_all['pred_month'] = train_clinical_all['visit_month']

for plus_month in [6, 12, 24]:
    train_shift = train_clinical_all[['patient_id', 'visit_month', 'pred_month', 'updrs_1', 'updrs_2', 'updrs_3', 'updrs_4']].copy()
    train_shift['visit_month'] -= plus_month
    train_shift.rename(columns={f'updrs_{i}': f'updrs_{i}_plus_{plus_month}' for i in range(1, 5)}, inplace=True)
    train_shift.rename(columns={'pred_month': f'pred_month_plus_{plus_month}'}, inplace=True)
    train_clinical_all = train_clinical_all.merge(train_shift, how='left', on=['patient_id', 'visit_month'])

train_clinical_all.rename(columns={f'updrs_{i}': f'updrs_{i}_plus_0' for i in range(1, 5)}, inplace=True)
train_clinical_all.rename(columns={'pred_month': f'pred_month_plus_0'}, inplace=True)
train_clinical_all

,visit_id,patient_id,visit_month,updrs_1_plus_0,updrs_2_plus_0,updrs_3_plus_0,updrs_4_plus_0,upd23b_clinical_state_on_medication,source,pred_month_plus_0,...,pred_month_plus_12,updrs_1_plus_12,updrs_2_plus_12,updrs_3_plus_12,updrs_4_plus_12,pred_month_plus_24,updrs_1_plus_24,updrs_2_plus_24,updrs_3_plus_24,updrs_4_plus_24
0,55_0,55,0,10.0,6.0,15.0,NaN,NaN,standard,0,...,12.0,10.0,10.0,41.0,0.0,24.0,16.0,9.0,49.0,0.0
1,55_6,55,6,8.0,10.0,34.0,NaN,NaN,standard,6,...,18.0,7.0,13.0,38.0,0.0,30.0,14.0,13.0,49.0,0.0
2,55_12,55,12,10.0,10.0,41.0,0.0,On,standard,12,...,24.0,16.0,9.0,49.0,0.0,36.0,17.0,18.0,51.0,0.0
3,55_18,55,18,7.0,13.0,38.0,0.0,On,standard,18,...,30.0,14.0,13.0,49.0,0.0,42.0,12.0,20.0,41.0,0.0
4,55_24,55,24,16.0,9.0,49.0,0.0,On,standard,24,...,36.0,17.0,18.0,51.0,0.0,48.0,17.0,16.0,52.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4501,65303_36,65303,36,4.0,1.0,26.0,0.0,NaN,supplemental,36,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4502,65382_0,65382,0,NaN,NaN,0.0,NaN,NaN,supplemental,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4503,65405_0,65405,0,5.0,16.0,31.0,0.0,NaN,supplemental,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4504,65530_0,65530,0,10.0,6.0,24.0,0.0,NaN,supplemental,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Find the best trend

In [7]:
def calculate_predicitons(pred_month, trend):
    if target == 'updrs_4': 
        pred_month = pred_month.clip(54, None)
    return np.round(trend[0] + pred_month * trend[1])

def function_to_minimize(x):    
    metric = smape_plus_1(
        y_true=y_true_array, 
        y_pred=calculate_predicitons(
            pred_month=pred_month_array,
            trend=x
        )
    )
    return metric

target_to_trend = {}
for i in range(1, 5):
    target = f'updrs_{i}'
    columns_with_target = [f'{target}_plus_{plus_month}' for plus_month in [0, 6, 12, 24]]
    columns_with_pred_month = [f'pred_month_plus_{plus_month}' for plus_month in [0, 6, 12, 24]]
    y_true_array = train_clinical_all[columns_with_target].values.ravel()
    pred_month_array = train_clinical_all[columns_with_pred_month].values.ravel()
    trend = list(minimize(
        fun=function_to_minimize,
        x0=[0, 0.0048],
        method='Powell'
    ).x)
    target_to_trend[target] = trend
    
target_to_trend

{'updrs_1': [5.394793062665313, 0.027091086167821344],
 'updrs_2': [5.469498130092747, 0.02824188329658148],
 'updrs_3': [21.182145576879183, 0.08897763331790556],
 'updrs_4': [-4.434453480103724, 0.07531448585334258]}

In [8]:
# It's possible to find the minimum (not guaranteed) of any other function with this approach
# (replace the function_to_minimize and x can be an array)

`target_to_tred` is our "model" which basically consists of the best linear model (`pred_month` as the only feature) for our train data for each **y_true** (updrs_1, updrs_2, updrs_3, updrs_4).

## Prediction

In [9]:
amp_pd_peptide.make_env.func_dict['__called__'] = False
env = amp_pd_peptide.make_env()   # initialize the environment
iter_test = env.iter_test()    # an iterator which loops over the test files

# The API will deliver four dataframes in this specific order:
for test_clinical_data, test_peptides, test_proteins, sample_submission in iter_test:
    sample_submission['patient_id'] = sample_submission['prediction_id'].map(lambda x: int(x.split('_')[0]))
    sample_submission['visit_month'] = sample_submission['prediction_id'].map(lambda x: int(x.split('_')[1]))
    sample_submission['target_name'] = sample_submission['prediction_id'].map(lambda x: 'updrs_' + x.split('_')[3])
    sample_submission['plus_month'] = sample_submission['prediction_id'].map(lambda x: int(x.split('_')[5]))
    sample_submission['pred_month'] = sample_submission['visit_month'] + sample_submission['plus_month']
    
    for i in range(1, 5):
        target = f'updrs_{i}'
        mask_target = sample_submission['target_name'] == target
        sample_submission.loc[mask_target, 'rating'] = calculate_predicitons(
            pred_month=sample_submission.loc[mask_target, 'pred_month'],
            trend=target_to_trend[target]
        )
        
    # call the env.predict for every iteration
    env.predict(sample_submission[['prediction_id', 'rating']])

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.
